In [1]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json

from functions import (get_sector, changes_from_press, get_df, 
                       normalise_to_index, save_data)

columns = ['date', 'press title', 'url', '1d change', 'index_price','norm_price']
past_last_date = False

C:\Users\Aidan\miniconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = []
page_num = 0

ticker = 'vrtx'

index_ticker = '^GSPC'
# select by value 
sector = get_sector(ticker)

stock_hist = get_df(ticker)
index_hist = get_df(index_ticker)

while True:
    
    x = requests.get(f'https://investors.vrtx.com/press-releases?page={page_num}')
    soup = BeautifulSoup(x.text)
    
    press_table = soup.find('table', attrs={'nirtable views-table views-view-table cols-3 collapse-table'})
    if press_table == None:
        break
    
    table_body = press_table.find('tbody')
    rows = table_body.find_all('tr')
    
    for row in rows:
        date = row.find('time').text
        date = datetime.strptime(date, '%b %d, %Y').date()
        if date < datetime(2014, 1, 1).date():
            past_last_date = True
            break
        title_url = row.find('td', attrs={'class': 'views-field views-field-field-nir-news-title'})
                             
        title = title_url.find('a').text
        url = title_url.find('a').get('href')
        url = f'https://investors.vrtx.com{url}'
        
        content = row.find('summary').text.replace('\n', '')

        pct_change = changes_from_press(stock_hist, date, 1)
        if pct_change != None:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
        else:
            index_price, norm_price = (None, None)
        data.append([str(date), title, url, content, pct_change, index_price, norm_price])

    if past_last_date == True: break
    page_num = page_num + 1
    
    
save_data(data, ticker, sector)